Sodelujoči

    Tomaž Štrus
    Anže Košir
    Jan Harej

O projektu: Iz platforme Steam, namenjenu predvsem za distrubicijo video iger, bomo zbrali podatke igrah več ljudi, ter analizirali stvari kot so npr. koliko iger, ki jih kupijo dejansko preigrajo, kolika časa bo nekdo zapravil za neko igro, kakšno igro bo kupil naslednjo, glede na svoje igre ipd.

Podatki: Uporabili bomo API, ki ga ponuja Steam: https://developer.valvesoftware.com/wiki/Steam_Web_API#JSON Tu lahko dobimo podatko o čisto vsemu kar potrebujemo npr. o uporabniku, njegove igre, prijatelji, koliko ur je igral neko igro, ... Dobili bomo podatke za npr. 100.000+ uporabnikov (prijatelji naših prijateljev, in njihov prijateljev itd.)

## Vprašanja za raziskavo
1. Povprečno dokončanje igre
2. Število in vrsta Bannov v povezavi z javnimi/privatnimi profili
3. Najmanjkrat odklenjeni dosežki za igrane igre

In [1]:
#install libraries on windows
'''
!python -m pip install numpy
!python -m pip install matplotlib
!python -m pip install requests
'''

'\n!python -m pip install numpy\n!python -m pip install matplotlib\n!python -m pip install requests\n'

In [2]:
import requests
import json
import numpy
import datetime
import os
from inspect import getouterframes, currentframe

In [3]:
api_key = "5F5DD2FA8A6C8646FCFE265C07BB90E5"
global_player_ids = set()
players_summaries = dict()
my_id="76561198101569818"

In [4]:
def get_player_summary (id):
    try:
        data = requests.get( "http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key="+api_key+"&steamids="+id).json()
    except:
        return None
    return data["response"]["players"][0]
    
def get_multiple_player_summary (ids):
    request_str = "http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key="+api_key+"&steamids="
    for id in ids:
        request_str += id+","
    request_str = request_str.strip(",")
    #print(request_str)
    try:
        data = requests.get(request_str).json()
    except:
        return None
    return data["response"]["players"]
    

In [5]:
get_player_summary("76561198202423736")
get_multiple_player_summary(["76561198202423736", "76561198012850892"])

[{'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/55/55e86a3f351cf10eefc1699352607f2140fc26ea.jpg',
  'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/55/55e86a3f351cf10eefc1699352607f2140fc26ea_full.jpg',
  'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/55/55e86a3f351cf10eefc1699352607f2140fc26ea_medium.jpg',
  'communityvisibilitystate': 3,
  'lastlogoff': 1523311930,
  'loccountrycode': 'SI',
  'personaname': 'VenomBlades',
  'personastate': 0,
  'personastateflags': 0,
  'primaryclanid': '103582791432579087',
  'profilestate': 1,
  'profileurl': 'http://steamcommunity.com/id/VenomBlades/',
  'steamid': '76561198012850892',
  'timecreated': 1250363859},
 {'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb.jpg',
  'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7f

In [6]:
def get_player_friends(id):
    try:
        data = requests.get( "http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key="+api_key+"&steamid="+id+"&relationship=friend").json()
    except:
        return None
    return data

In [7]:
get_player_friends(my_id)

{'friendslist': {'friends': [{'friend_since': 1396456108,
    'relationship': 'friend',
    'steamid': '76561197992822334'},
   {'friend_since': 1508496479,
    'relationship': 'friend',
    'steamid': '76561198007584342'},
   {'friend_since': 1447627152,
    'relationship': 'friend',
    'steamid': '76561198009469751'},
   {'friend_since': 1396454834,
    'relationship': 'friend',
    'steamid': '76561198012850892'},
   {'friend_since': 1440937046,
    'relationship': 'friend',
    'steamid': '76561198019143410'},
   {'friend_since': 1396519168,
    'relationship': 'friend',
    'steamid': '76561198021259953'},
   {'friend_since': 1482788739,
    'relationship': 'friend',
    'steamid': '76561198023306579'},
   {'friend_since': 1392400547,
    'relationship': 'friend',
    'steamid': '76561198033366594'},
   {'friend_since': 1390342420,
    'relationship': 'friend',
    'steamid': '76561198043441963'},
   {'friend_since': 1390129991,
    'relationship': 'friend',
    'steamid': '76561

In [8]:
def fill_player_ids_rec(id, num = 1000, already_called = set()):
    global global_player_ids
    already_called.add(id)
    print("recursive call of id: " + id + "  len: " + str(len(global_player_ids)))
    try:
        if get_player_summary(id)["profilestate"] != 1:
            #print("not public")
            return None
    except KeyError:
        #print("not public key error")
        return None
    try:
        data = get_player_friends(id)["friendslist"]["friends"]
    except KeyError:
        #print("no friends")
        return None
    tmp_id = set()
    for friend in data:
        if len(global_player_ids) >= num:
            print("enough friends")
            break
        tmp_id.add(friend["steamid"])
        global_player_ids.add(friend["steamid"])
        #print("len: "+ str(len(global_player_ids)))
        
    print("added my friends" + "  len: " + str(len(global_player_ids)))
    
    for playerid in tmp_id:
        if len(global_player_ids) >= num:
            break
        try:
            data = get_player_friends(playerid)["friendslist"]["friends"]
        except KeyError:
            pass
            #print("no friends")
        for fof in data:
            if len(global_player_ids) >= num:
                print("enough friends")
                break
            global_player_ids.add(fof["steamid"])
        #print("added friends of friend")
            
    print("added friends of friends" + "  len: " + str(len(global_player_ids)))
    
    for playerid in tmp_id:
        if len(global_player_ids) >= num:
            print("enough friends")
            break
        if playerid not in already_called:
            fill_player_ids_rec(playerid, num, already_called)
    
    

In [9]:
def get_players_summaries():
    print("halo")
    global global_player_ids
    global players_summaries
    id_buffer = set()
    
    for player_id in global_player_ids:
        id_buffer.add(player_id)
        if len(id_buffer) >= 100:
            summaries = get_multiple_player_summary(id_buffer)
            for summary in summaries:
                players_summaries[summary["steamid"]] = summary
            id_buffer = set()
            print("summaries len: " + str(len(players_summaries)))
    
    summaries = get_multiple_player_summary(id_buffer)
    
    for summary in summaries:
        players_summaries[summary["steamid"]] = summary
    id_buffer = set()

In [10]:
#--------------- request player ids from api ------------------
#global_player_ids = set()
#fill_player_ids_rec(my_id, 10000)

#--------------- request player ids from file -----------------
with open("pid_set.json", "r") as fp:
    global_player_ids = set(json.load(fp))
    
#------------- request player summary from api ----------------
#players_summaries = dict()
#get_players_summaries()

#------------- request player summary from file ---------------
with open("player_summaries.json", "r") as fp:
    global_player_ids = json.load(fp)


global_player_ids
print(len(global_player_ids))

#----------------- write player ids to file -------------------
with open("pid_set.json", "w") as fp:
    json.dump(list(global_player_ids), fp)
    
#--------------- write player summary to file -----------------
with open("player_summaries.json", "w") as fp:
    json.dump(players_summaries, fp)

10000
